# Replication of @zhang_flexibleimplementationtrilinearity_2022 Rank Estimation

This study replicated the reported results of @zhang_flexibleimplementationtrilinearity_2022 on a Wine GC-MS dataset. The dataset is described in section 3.2 and the results in 4.2.

We first recreated the dataset and developed methods to preprocess, unfold, decompose and display the estimated singular values.

## Dataset

The GC-MS data was downloaded from [here](https://ucphchemometrics.com/2023/06/01/wine-samples-analyzed-by-gc-ms-and-ft-ir-instruments/) and originally prepared by @skov_multiblockvariancepartitioning_2008. It is stored in MATLAB 7.3 format and required the use of [mat73](https://gitlab.com/obob/pymatreader/) library. Within the library is GC-MS, FT-I and physicochemical univariate measurements. The GC-MS data consists of 44 samples x 2700 elution time-points and 200 mass channels.

The authors narrowed the scope to range from 16.52 to 16.76 mins (corresponding to a range of 25 units) containing two compounds (peaks), described in detail in section 3.2. They identified three significant components (chemical rank) attributing two to the compounds stated and one to the background. We expect to find the same.

In [ ]:
%reload_ext autoreload
%autoreload 3

import numpy as np
from pymatreader import read_mat
import xarray as xr
from pca_analysis.notebooks.experiments.zhang_pca_scree_plot import PCA
from pca_analysis.definitions import DATA_DIR
from pca_analysis import xr_plotly

xr.set_options(display_expand_data=False)

mat_path = DATA_DIR / "Wine_v7.mat"
nc_path = DATA_DIR / "Wine_v7.nc"

if nc_path.exists():
    full_data = xr.open_dataarray(nc_path)

else:
    mat_data = read_mat(mat_path)

    key_it = [
        "Label_Wine_samples",
        "Label_Elution_time",
        "Label_Mass_channels",
    ]

    full_data = xr.DataArray(
        mat_data["Data_GC"],
        dims=["sample", "time", "mz"],
        coords=[mat_data[k] for k in key_it],
    )

    full_data.name = "zhang_data"
    full_data.to_netcdf(nc_path)


# need number of time cells to be greater than the predicted rank (22)
zhang_demo_intvl = full_data.sel(
    time=slice(16.52 - 0.08, 16.76 + 0.08)
)  # interval taken from article
display(zhang_demo_intvl)

(
    zhang_demo_intvl.sel(mz=slice(0, 100))
    .isel(sample=slice(0, 44, 4))
    .plotly.facet(plot_type="heatmap", x="mz", y="time", z="sample", n_cols=3)
)


The data interval is displayed above and corresponds to that shown by @zhang_flexibleimplementationtrilinearity_2022 figure 5. The 2 peaks are best viewed at mz = 44, as below:

In [ ]:
zhang_demo_intvl.sel(mz=44).plotly.line(x="time", color="sample")


In [ ]:
(
    zhang_demo_intvl.isel(mz=slice(40, 60))
    .isel(sample=slice(0, 44, 4))
    .plotly.facet(n_cols=2, x="time", y="mz", z="sample", plot_type="line")
)

# .hvplot(by="sample", alpha=0.5)


Where we can see that the peaks are present in all samples at varying intensities.


## Preprocessing



### Unfolding 

@zhang_flexibleimplementationtrilinearity_2022 in section 2.5 describe the application of SVD (PCA) to different unfoldings of the data tensor. They state that depending on which mode is unfolded, the SVD will produce different numbers of significant components if the data is not perfectly trilinear, otherwise all three unfoldings will have the same number of significant components. They state that $X_{\text{caug}}$ produces the most accurate estimate of significant components in the face of noise and trilinear disruption, assuming that each chemical species has a unique spectrum and their relative concentrations are independent. What is $X_{\text{caug}}$? It is the unfolding $(I \times K, J)$, which in the context of the dataset is $(\text{retention times} \times \text{mz}, \text{samples})$. Thus we first need to produce the augmented (unfolded) matrix $C_\text{aug}$, unfolding along the sample mode.


In [ ]:
from sklearn_xarray.preprocessing import BaseTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def unfold(x):
    return x.stack({"aug": ("sample", "time")}).transpose(..., "mz")


class Unfolder(BaseTransformer):
    def __init__(self):
        # required for API
        self.groupby = None
        pass

    def _transform(self, X):
        unfolded = unfold(X)
        return unfolded


pipeline = Pipeline([("unfold", Unfolder()), ("pca", PCA())])
pipeline.fit_transform(zhang_demo_intvl)
pipeline.named_steps["pca"].scree_plot()


## Estimating the number of Components

While @zhang_flexibleimplementationtrilinearity_2022 specify the use of the SVD, a useful interface for estimating the number of components is sklearn's `PCA`. A rudimentary scree plot can be used to observe the inflection point described in figure 3. The authors state in section 4.1 that when observing a function of the explained variance against the number of components, the point where the explained variance does not "change much anymore" is the point where the components start describing the noise of the dataset rather than chemical species.

And as we can see, we're able to recreate the results if the cutoff of the magnitude of change is set to 0.005.

## Full Dataset

Now we'll see what happens if the same method is applied to the full dataset, as its only valid if it works for any number of peaks. If not then some underlying mechanism is at work. To reiterate, we're expecting the number of significant components == the number of peaks == the number of unique chemical species.

### Visual Estimation of Number of Peaks

What are the number of components expected? It should be close to the number of peaks in the maximum mass channel. This is because its a fair assumption that the sample with the most abundant chemical species is also the most intense. To find this value we will find the mass channel with the highest amplitude then the sample with the highest average amplitude at that mass channel. That sample and mz is displayed below, with its peaks highlighted in red.

In [ ]:
# average mz over time and samples

mean_max_mz_label = full_data.mean("time").mean("sample").idxmax().pipe(int)

max_sample_label = full_data.isel(mz=mean_max_mz_label).mean("time").idxmax().item()

max_sample = full_data.sel(sample=max_sample_label, mz=mean_max_mz_label)
display(max_sample.sel(time=slice(4, 24)).plotly.line(x="time"));


In [ ]:
from pca_analysis.peak_picking import compute_dataarray_peak_table


pt = compute_dataarray_peak_table(
    xa=zhang_demo_intvl.transpose("sample", "mz", "time")
    .isel(sample=slice(0, 5))
    .sel(mz=slice(40, 48))
)
ds = xr.merge([pt, zhang_demo_intvl])
ds


In [ ]:
from pca_analysis.notebooks.experiments.zhang_experiment_find_peaks import FindPeaks

fp = FindPeaks()
fp.find_peaks(sample=max_sample)
display(fp.plot_peaks(sample=max_sample))
n_peaks = len(fp.peaks_x)

display(f"{n_peaks=}")


TODO continue revising, cleaning, etc.


In [ ]:
from IPython.display import Markdown

Markdown(
    f"Visually it appears that all peaks are accounted for, thus the number of peaks is around {n_peaks}. Thus we should expect around {n_peaks} significant components through PCA."
)


### Unfold Full Dataset

In [ ]:
pipeline.fit_transform(full_data)
pca = pipeline.named_steps["pca"]
pca.scree_plot()


But as we can see, the profile is coincidentally similar to the 2 peak slice with the vast majority of the variance explained by the first three compoonents, We can presume that this is because of the dominance of the maxima peak. If we remove it from the set..

## Full Dataset Without Maxima

In [ ]:
def exclude_global_maxima(xa: xr.DataArray, peaks_x: np.ndarray, peaks_y: np.ndarray):
    """
    select the subset of the input signal running from half way between the maxima and
    the next peak until the end of the signal
    """
    maxima_idx = peaks_y.argmax()
    next_peak_idx = maxima_idx + 1

    # maxima idx plus the next idx minus the maxima idx.
    # the time half way between the two.
    maxima_time = peaks_x[maxima_idx]
    next_peak_time = peaks_x[next_peak_idx]

    cut_time_start = maxima_time + ((next_peak_time - maxima_time) / 2)

    without_global_maxima = xa.sel(time=slice(cut_time_start, None))

    return without_global_maxima


without_global_maxima = full_data.pipe(
    exclude_global_maxima, peaks_x=fp.peaks_x, peaks_y=fp.peaks_y
)
without_global_maxima.pipe(display)
without_global_maxima.sel(mz=44).isel(sample=slice(0, 44, 11)).plotly.line(
    x="time", facet_col="sample", facet_col_wrap=2
)


Looks good. Whats the PCA?

In [ ]:
pipeline = Pipeline([("unfold", Unfolder()), ("pca", PCA())])
pipeline.fit_transform(without_global_maxima)
pipeline.named_steps["pca"].scree_plot()


An improvement, but evidently the PCA is still being dominated by a subset of latent variables. A possible solution is in standard scaling...

## Scaled and Centered PCA Global Maxima


Scaling and centering can help models such as PCA fit noisy or otherwise abbhorant data. In this context, where after unfolding the dataset is represented by samples (sample and timewise rows) and features (spectral dimension columns), then centering is the subtraction of each columns mean from each row and scaling is the division of each by the columns standard variation. This is implemented by `sklearn`'s `StandardScaler`. The result is that each column ranges from 0 to 1 and has a mean of 0.

In [ ]:
pipeline_with_std_scl = Pipeline(
    [
        ("unfold", Unfolder()),
        ("scale", StandardScaler()),
        ("pca", PCA()),
    ]
)

pipeline_with_std_scl.fit_transform(full_data)
pca2 = pipeline_with_std_scl.named_steps["pca"]
pca2.scree_plot()


A reversal in results. It appears that PCA with this user defined metric estimates an optimal number of components equal to the number of peaks detected earlier. Evidently standard scaling the data meaningfully increases the number of significant components estimated by the scree plot. This indicates that it is an appropriate method of estimation for unaligned GC-MS data. To further explore the veracity of the result we would need to estimate the total number of unique peaks across the sample dimension, as this verification has only observed the sample with the absorbance maxima.

## CORCONDIA

CORCONDIA is a method of approximating the number of components of PARAFAC and PARAFAC-like models (i.e. PARAFAC2) through observation of a Tucker3 core [@bro_newefficientmethod_2003]. The algorithm iterates through components, starting at 1, until a user-specified limit, and we are looking for a steep dropoff as the indication of the optimal number of components. In the interest of speed we will restrict the signal to the 15 to 25 minute interval.

In [ ]:
from corcondia import corcondia_3d

display(zhang_demo_intvl)
display(
    zhang_demo_intvl.sel(
        mz=44,
    )
    .isel(sample=slice(0, 44, 11))
    .plotly.line(x="time", color="sample")
)


In [ ]:
class CORCONDIA:
    def __init__(self, X):
        if not isinstance(X, np.ndarray):
            raise TypeError
        self.X = X
        self.diagnostics = None
        self.ranks = None
        pass

    def compute_range(self, rank_range=(1, 2)):
        """
        calculate the CORCONDIA diagonstic for a range of ranks
        """

        self.diagnostics = []
        self.ranks = []
        for k in range(rank_range[0], rank_range[1] + 1):
            self.ranks.append(k)
            self.diagnostics.append(self.compute_diagnostic(rank=k))

    def compute_diagnostic(self, rank):
        """
        compute the diagonistic for a given rank
        """

        assert isinstance(rank, int)
        assert rank > 0
        return corcondia_3d(X=self.X, k=rank)

    def plot_diagonstic(self):
        """
        viz the diagnostic range generated by `self.compute_range`
        """

        if (self.diagnostics is None) or (self.ranks is None):
            raise RuntimeError("run `compute_range` first")

        fig = go.Figure()

        fig.add_trace(go.Scatter(x=self.ranks, y=self.diagnostics))
        fig.update_layout(
            title="CORCONDIA Diagnostic vs. Model Rank",
            xaxis=dict(title="rank"),
            yaxis=dict(title="diagnostic"),
        )
        return fig


cc = CORCONDIA(X=zhang_demo_intvl.to_numpy())
cc.compute_range(rank_range=(1, 10))
cc.plot_diagonstic()


As we can see, the CORCONDIA results are ambiguous over a wide range but indicate that a rank of 3 maintains model stability, which is in agreement with the PCA results. Note that while we havent' demonstrated it here ,the results are semi-random and that deserves more study.

## PARAFAC2

### PARAFAC2 Demo Dataset

A demonstration of PARAFAC2 on the demo dataset. We demonstrated earlier that a rank of 3 is appropriate.


In [ ]:
zhang_demo_intvl.sel(mz=44).plotly.line(x="time", color="sample")


In [ ]:
from pca_analysis.parafac2_pipeline.estimators import PARAFAC2
from pca_analysis import xr_plotly  # noqa

rank_range = 3

parafac2_pipeline = Pipeline(
    [
        (
            "parafac2",
            PARAFAC2(rank=rank_range, n_iter_max=500, nn_modes="all", linesearch=False),
        )
    ]
)

parafac2_pipeline.fit_transform(zhang_demo_intvl)

parafac2 = parafac2_pipeline.named_steps["parafac2"]
results = parafac2.results_as_xr()

components = results.components


In [ ]:
components.sel(mz=44).transpose("component", "sample", ...).plotly.facet(
    x="time", y="sample", z="component", n_cols=2
).update_layout(
    height=500, title="samples per component of the Zhang et. al demo interval"
)


This result corresponds to the deconstruction shown in @zhang_flexibleimplementationtrilinearity_2022, Figure 6.

TODO make viz better.


### PARAFAC2 Full Dataset

We demonstrated earlier that through the PCA approach, we estimate 22 components for the full dataset.


The decomposition of the full dataset with rank = 22 and n_iter_max = 500 takes 7m 25s which is too slow to retain for testing purposes. Furthermore the results are disappointing, with some components capturing multiple peaks while others capture nothing.

In [ ]:
# load results from cache if the file is present, saving time.

full_data_reults_path = DATA_DIR / "zhang_full_data_parafac2_decomp_results.nc"
if not full_data_reults_path.exists():
    parafac2_pipeline.set_params(parafac2__rank=22).fit_transform(full_data)
    parafac2 = parafac2_pipeline.named_steps["parafac2"]
    results = parafac2.results_as_xr()

    results.to_netcdf(full_data_reults_path, engine="netcdf4")


else:
    results = xr.open_dataset(full_data_reults_path)


In [ ]:
xa = (
    results.components[0:6]
    .sel(mz=44)
    .isel(component=slice(0, 4))
    .transpose("component", "sample", ...)
)
xa.name = "abs"
n_cols = 2
x = "time"

fig = xa.plotly.facet(x="time", y="sample", z="component", n_cols=2)
fig.update_layout(title="sample overlays per component")
fig


As we can see from the viz above, the decomposition is far from perfect, with some components capturing multiple peaks and many others capturing nothing. Furthermore there are multiple negative peaks, which is indicative of a very poor model. Further research will be required to explain these results.

## Conclusion

While I was able to successfully recreate Zhang's results and demonstrate that the scree test appears to be able to predict the number of components well when appropriate scaling is used. We found that CORCONDIA has random results which warrent more investigation, and that while PARAFAC performs admirably on a small number of peaks, it fails dramatically when applied to more complicated data.